In [15]:
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
import time
from IPython.display import display

In [10]:
#pip install requests beautifulsoup4 folium geopy

     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
   ---------------------------------------- 125.4/125.4 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [27]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Path to your WebDriver (e.g., ChromeDriver)
webdriver_path = 'D:/s9/Concertify/chromedriver-win64/chromedriver.exe'

# Initialize a Selenium WebDriver
driver = webdriver.Chrome(webdriver_path)

# Go to the Ticketmaster Paris page
driver.get('https://www.ticketmaster.fr/fr/paris')
time.sleep(5)  # Wait for JavaScript to load the content

# Get the page source and close the browser
html = driver.page_source
driver.quit()

# Parse the page with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

concerts = []
for i in range(5):
    event = soup.find('li', id=f'event-result-{i}')
    if event:
        title = event.find('h3', class_='event-result-title').get_text(strip=True)
        date = event.find('time', itemprop='startDate').get_text(strip=True)

        # Extract the arena name
        arena = event.find('span', itemprop='name').get_text(strip=True) if event.find('span', itemprop='name') else "Unknown Arena"

        # Extract the city location
        city_location = event.find('span', itemprop='addressLocality').get_text(strip=True) if event.find('span', itemprop='addressLocality') else "Unknown City"

        location = f"{arena}, {city_location}"
        concerts.append({"title": title, "location": location, "date": date})


# Print the concert information
for concert in concerts:
    print(concert)


{'title': 'DJ SNAKE', 'location': 'ACCOR ARENA, PARIS', 'date': '10 mai 2025'}
{'title': 'DOJA CAT', 'location': 'ACCOR ARENA, PARIS', 'date': '21 juin 2024'}
{'title': 'PACKAGE DOJA CAT', 'location': 'ACCOR ARENA, PARIS', 'date': '21 juin 2024'}
{'title': 'DOJA CAT', 'location': 'ACCOR ARENA, PARIS', 'date': '21 juin 2024'}
{'title': 'KAROL G', 'location': 'ACCOR ARENA, PARIS', 'date': '22 juin 2024'}


In [28]:
def geocode_location(location):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        loc = geolocator.geocode(location)
        return (loc.latitude, loc.longitude)
    except:
        return None


# Geocode each concert location
for concert in concerts:
    coords = geocode_location(concert['location'])
    if coords:
        concert['coords'] = coords
    else:
        concert['coords'] = (0, 0)  # Default coords, adjust as necessary
    time.sleep(1)  # To avoid hitting request limits on the geocoding API

# Create a map and add markers
map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)  # Centered on Paris
for concert in concerts:
    folium.Marker(
        location=concert['coords'],
        popup=f"{concert['title']} - {concert['date']}",
        tooltip=concert['title']
    ).add_to(map)

In [30]:
display(map)